In [1]:
# Eksempel 5.2 i bogen Model Building in Mathematical Programming
# Transportproblem
# Strukturen i nedenstående svarer til dette eksempel i PuLP-dokumentationen med mindre modifikationer:
# https://coin-or.github.io/pulp/CaseStudies/a_transportation_problem.html

# Importer PuLP-funktioner
import pulp as PLP

# Minimeringsproblem
Model = PLP.LpProblem("Eks52",PLP.LpMinimize)

UdbyderRange = range(1,5) # 1..4
Supply = [0,400,200,300, 100]

KundeRange = range(1,6) # 1..5
Demand = [0,225, 125, 150, 325, 175]

M = 999 # meget dyr derfor vælge M ikke i den optimale løsning
UnitCosts = [[0],
             [0, 3, 6, 6, 7, 7],
             [0, 2, 9, 4, 9, 4],
             [0, 8, 8, 3, 1, 7],
             [0, 5, 4, 8, 5, 2]]

# Beslutningsvariable x(i,j)
x = PLP.LpVariable.dicts("x",(UdbyderRange,KundeRange),0,None) # x(1,1) .. x(3,4)

# Delta(i,j)
Delta = PLP.LpVariable.dicts("Delta",(UdbyderRange,KundeRange),cat='Binary')

# Binær variabel som bliver 1 hvis F1 sender til mere end 2 kunder
z = PLP.LpVariable("F1_Extra_Cost", cat='Binary')

# Opdater objektfunktionen til at inkludere ekstra fast omkostning
Model += 100 * z + PLP.lpSum(UnitCosts[i][j] * x[i][j] for i in UdbyderRange for j in KundeRange), "Objektfunktion"

# Udbud:
for i in UdbyderRange:
    Model += PLP.lpSum([x[i][j] for j in KundeRange]) <= Supply[i], f"Supply_{i}"

# Efterspørgsel:
for j in KundeRange:
    Model += PLP.lpSum([x[i][j] for i in UdbyderRange]) == Demand[j], f"Demand_{j}"

# Aktiver Delta hvis x[i][j] > 0
for i in UdbyderRange:
    for j in KundeRange:
        Model += x[i][j] <= M * Delta[i][j], f"Delta_Activation_{i}_{j}"

# Begrænsning: z = 1 hvis F1 sender til mere end 2 kunder
Model += PLP.lpSum([Delta[1][j] for j in KundeRange]) <= 2 + 10 * z, "ExtraCostTrigger"

# Løs modellen
Model.solve(PLP.PULP_CBC_CMD(msg=False))

# Print resultat
print("Status:", PLP.LpStatus[Model.status])
for v in Model.variables():
    print(v.name, "=", v.varValue)
print("Obj. = ", PLP.value(Model.objective))

Status: Optimal
Delta_1_1 = 1.0
Delta_1_2 = 1.0
Delta_1_3 = 1.0
Delta_1_4 = 1.0
Delta_1_5 = 1.0
Delta_2_1 = 1.0
Delta_2_2 = 1.0
Delta_2_3 = 1.0
Delta_2_4 = 1.0
Delta_2_5 = 1.0
Delta_3_1 = 1.0
Delta_3_2 = 1.0
Delta_3_3 = 1.0
Delta_3_4 = 1.0
Delta_3_5 = 1.0
Delta_4_1 = 1.0
Delta_4_2 = 1.0
Delta_4_3 = 1.0
Delta_4_4 = 1.0
Delta_4_5 = 1.0
F1_Extra_Cost = 1.0
x_1_1 = 225.0
x_1_2 = 125.0
x_1_3 = 25.0
x_1_4 = 25.0
x_1_5 = 0.0
x_2_1 = 0.0
x_2_2 = 0.0
x_2_3 = 125.0
x_2_4 = 0.0
x_2_5 = 75.0
x_3_1 = 0.0
x_3_2 = 0.0
x_3_3 = 0.0
x_3_4 = 300.0
x_3_5 = 0.0
x_4_1 = 0.0
x_4_2 = 0.0
x_4_3 = 0.0
x_4_4 = 0.0
x_4_5 = 100.0
Obj. =  3150.0
